## HW4
Write a program to price European put options based on the GARCH model (Ritchken-Trevor algorithm).

- Inputs:
    - E (days before expiration)
    - r (%) (interest rate)
    - S0 (stock price at time 0)
    - h0, b0, b1, b2, c
    - X (strike price)
    - n1 (number of partitions per day)
    - n2 (number of variances per node)

- Output: price

> E = 30, r (%) = 5, S0 = 100  
> h0 = 0.010469, b0 = 0.000006575, b1 = 0.9, b2 = 0.04, c = 0  
> X = 100, n1 = 3, n2 = 3  

> Option price: about 2.0163

### GARCH Option Pricing
$$\ln \frac{S_{t+1}}{S_t} = r - \frac{h_t^2}{2} + h_t\epsilon_{t+1}$$
With $y_t \triangleq \ln S_t$,
$$y_{t+1} = y_t+ r - \frac{h_t^2}{2} + h_t\epsilon_{t+1}$$
where
$$h_{t+1}^2 = \beta_0 + \beta_1h_t^2+\beta_2h^2_t(\epsilon_{t+1}-c)^2$$
$$\epsilon \sim N(0,1) \text{ given information at date }t$$
$$r = \text{ daily riskless return}$$
$$c \geq 0$$

In [1]:
# import numpy as np
# days = 30   # days before expiration
# ar = 5   # annual interest rate
# S0 = 100
# h0 = 0.010469
# b0 = 0.000006575
# b1 = 0.9
# b2 = 0.04
# c = 0
# X = 100
# n_part = 3  # number of partitions per day
# n_var = 3   # number of variances per node

In [2]:
import numpy as np
days = input("Days before expiration: ")
ar = input("Annual interest rate (%): ")
S0 = input("Stock price at time 0: ")
h0 = input("h0: ")
b0 = input("b0: ")
b1 = input("b1: ")
b2 = input("b2: ")
c = input("c: ")
X = input("Strike price: ")
n_part = input("Number of partitions per day: ")
n_var = input("Number of variances per node: ")

Days before expiration: 30
Annual interest rate (%): 5
Stock price at time 0: 100
h0: 0.010469
b0: 0.000006575
b1: 0.9
b2: 0.04
c: 0
Strike price: 100
Number of partitions per day: 3
Number of variances per node: 3


### The Ritchken-Trevor (RT) Algorithm
- Partition a day into $n$ periods.
- Three states follow each state $(y_t, h^2_t)$ after a period. 
- Define $\gamma\triangleq h_0$,
$$\gamma_n \triangleq \frac{\gamma}{\sqrt{n}}$$
- Jump size $$\eta\gamma_n$$

- A few nodes may have multiple jump sizes because more than one paths can reach those node.
- Each node on the tree contains only two states $(y_t, h^2_{max})$ and $(y_t, h^2_{min})$, and each state carries its own $\eta$ and set of $2n + 1$ branching probabilities.

In [3]:
ar /= 100.
r = ar / 365 # daily interest rate
gamma = h0
gamma_n = h0 / (n_part**0.5)
branches = 2 * n_part + 1

# Probability Table
P = [{} for _ in range(days)]

# Eta Table
E  = [{} for _ in range(days)]

# Variance Table
H2  = [{} for _ in range(days+1)]
H2[0][0] = [h0**2]*n_var
    # H2[i][j][0] = h^2_{min} (i,j)
    # H2[i][j][n_var-1] = h^2_{max} (i,j)

### Eta
- The necessary requirement $p_m \geq 0$ implies $\eta \geq h_t/\gamma$
- Different conditional variances $h^2_t$ may require different $\eta$, hence we try 
$$\eta = \lceil h_t/\gamma\rceil, \lceil h_t/\gamma\rceil+1,\lceil h_t/\gamma\rceil+2,...$$
until valid probabilities are obtained or until their
nonexistence is confirmed.
- The sufficient and necessary condition for valid probabilities to exist is $$\frac{\mid r-(h_t^2/2)\mid}{2\eta\gamma\sqrt{n}} \leq \frac{h_t^2}{2\eta^2\gamma^2} \leq \min\Big(1-\frac{\mid r-(h_t^2/2)\mid}{2\eta\gamma\sqrt{n}}, \frac{1}{2}\Big)$$

In [4]:
def compute_eta(h2_t):
    eta = int(np.ceil((h2_t ** 0.5) / gamma))
    while True:
        lower_bound = abs(r - (h2_t/2)) / (2 * eta * gamma * n_part**0.5)
        middle = h2_t / (2 * eta**2 * gamma**2)
        upper_bound = min(1 - abs(r - (h2_t/2)) / (2 * eta * gamma * n_part**0.5), 0.5)
        if lower_bound <= middle and middle <= upper_bound: 
            return eta
        else:
            eta += 1

### Probability
$$p_u = \frac{h_t^2}{2\eta^2\gamma^2}+\frac{r-(h_t^2/2)}{2\eta\gamma\sqrt{n}}$$   $$p_m = 1-\frac{h_t^2}{\eta^2\gamma^2}$$
$$p_d = \frac{h_t^2}{2\eta^2\gamma^2}-\frac{r-(h_t^2/2)}{2\eta\gamma\sqrt{n}}$$
- $l$:  the number of up moves must exceed that of down moves
    - $-n \leq l \leq n$
    - $y_{t+1}=y_t+l\eta\gamma_n$
    - The probability
        $$P(l) \triangleq \sum\limits_{j_u, j_m, j_d}\frac{n!}{j_u! j_m! j_d!}p_u^{j_u}p_m^{j_m}p_d^{j_d}$$
        with $j_u, j_m, j_d \geq 0, m = j_u+j_m+j_d \text{ and } l = j_u -j_d$
- Calculate $P(l)$
$$(p_ux+p_m+\frac{P_d}{x})^n = \sum\limits_{l=-n}^{n}P(l)x^l$$
$$\Rightarrow (p_ux^2+p_mx+P_d)^n = \sum\limits_{l=-n}^{n}P(l)x^{l+n} = \sum\limits_{l=0}^{2n}P(l)x^{l}$$

In [5]:
def compute_P(h2_t, eta, P, i, j, k):
    pu = h2_t / (2 * eta**2 * gamma**2) + (r - (h2_t/2)) / (2 * eta * gamma * n_part**0.5)
    pd = h2_t / (2 * eta**2 * gamma**2) - (r - (h2_t/2)) / (2 * eta * gamma * n_part**0.5)
    pm = 1 - h2_t / (eta**2 * gamma**2)
    
    c = np.polynomial.polynomial.polypow([pd, pm, pu], n_part) # (coefficient: low->high, power)
    for l in range(branches):
        P[i][j][k][l] = c[l]

### Variance
- The logarithmic price $y_t + l\eta\gamma n$ at date $t+1$ following state $(y_t, h^2_t)$ is associated with the variance
$$h_{t+1}^2 = \beta_0 + \beta_1h_t^2+\beta_2h^2_t(\epsilon'_{t+1}-c)^2$$
where $$\epsilon'_{t+1} = \frac{l\eta\gamma_n-(r-h_t^2/2)}{h_t}, l = 0, \pm 1, \pm 2,...\pm n$$


     
- Besides the minimum and maximum variances, the other K − 2 variances in between are linearly interpolated.
- In general, the $k$th variance at node $(i, j)$ is
$$h^2_{min}(i,j)+k\frac{h^2_{max}(i,j)-h^2_{min}(i,j)}{K-1}, k = 0, 1, ..., K-1$$

In [6]:
def compute_max_min_var(h2_cur, eta, H2, i_next):
    for l in range(-n_part, n_part+1):
        epsilon =  (l * eta * gamma_n - (r - h2_cur / 2))  / (h2_cur ** 0.5)
        h2_next = b0 + b1 * h2_cur + b2 * h2_cur * (epsilon - c)**2
        j_next = j + l * eta
        if H2[i_next].has_key(j_next):
            H2[i_next][j_next][0] = min(H2[i_next][j_next][0], h2_next)
            H2[i_next][j_next][-1] = max(H2[i_next][j_next][-1], h2_next)
        else:
            H2[i_next][j_next] = [h2_next]*n_part
            
def compute_interpolation_var(H2, next_i):
    for j in  sorted(H2[next_i].keys()):
        h2_min = H2[next_i][j][0]
        h2_max = H2[next_i][j][-1]
        for k in range(n_var):
            H2[next_i][j][k] = h2_min + k * (h2_max - h2_min) / (n_var -1)

### Build RT Tree
- $\eta$ tree: ```E[days][keys][n_var]```
- $p$ tree: ```P[days][keys][n_var][branches]```
- $h^2$ tree: ```H2[days][keys][n_var]```

In [7]:
for i in range(days):
    for j in sorted(H2[i].keys()): 
        E[i][j] = [0] * n_var
        P[i][j] = [[0]*branches for _ in range(n_var)]
        for k in range(n_var):
            h2_cur = H2[i][j][k]
            eta = compute_eta(h2_cur)
            E[i][j][k] = eta
            compute_P(h2_cur, eta, P, i, j, k)
            # compute variance of day i+1 with variance of day i
            compute_max_min_var(h2_cur, eta, H2, i+1)
    # Interpolation
    compute_interpolation_var(H2, i+1)

### Put Option Price at expiration
- Stock price at day i  
$y(i,j) = y(0,0)+\gamma_nj$  
$\Rightarrow S(i,j) = S(0,0) e^{\gamma_nj}$
- Put Price Table: ```PP[days][keys][n_var]```

In [8]:
PP = [{} for _ in range(days+1)] # Put Price
for i in range(days+1):
    for j in sorted(H2[days].keys()):
        PP[i][j] = [max(X - S0 * np.exp(gamma_n * j), 0)] *n_var

### Option price - Interpolation 
- If the updated variance falls strictly between $h^2_{max}$ and $h^2_{min}$, use an interpolation to compute the corresponding option price.
- A variance following an interpolated variance may exceed the maximum variance or be exceeded by the minimum variance. When this happens, the option price corresponding to the maximum or minimum variance will be used during backward induction.

In [9]:
def compute_interpolation_price(next_i, next_j, h2_cur):
    # Next h^2
    epsilon =  (l * eta * gamma_n - (r - h2_cur / 2))  / (h2_cur ** 0.5)
    h2_next = b0 + b1 * h2_cur + b2 * h2_cur * (epsilon - c)**2
    if h2_next >  H2[next_i][next_j][-1]:  # > h^2_{max}
         return PP[next_i][next_j][-1]
    if h2_next < H2[next_i][next_j][0]:   # < h^2_{min}
        return PP[next_i][next_j][0]
    for next_k in range(n_var-1):
        h2_lower = H2[next_i][next_j][next_k] 
        h2_upper = H2[next_i][next_j][next_k+1]
        if h2_next >= h2_lower and h2_next <= h2_upper:
            if h2_lower == h2_upper:
                return PP[next_i][next_j][next_k]
            x = (h2_next - h2_lower) / (h2_upper - h2_lower)
            return (1-x) * PP[next_i][next_j][next_k] + x * PP[next_i][next_j][next_k+1]

### Backward Induction
$$Price(i, j, k) = \frac{\sum\limits_{l=-n}^{n}P(i,j,k,l)*Price(i+1, j+l\eta, k')}{e^r}$$
where $k'$ is computed by an interpolation between the two closest options.

In [10]:
for i in range(days-1, -1, -1):
    for j in  sorted(H2[i].keys()):
        for k in range(n_var):
            eta = E[i][j][k]
            h2_cur = H2[i][j][k]
            put = 0
            for l in range(-n_part, n_part+1): # branches
                prob = P[i][j][k][l+n_part]
                next_j = j + eta * l
                price = compute_interpolation_price(i+1, next_j, h2_cur)
                put += price * P[i][j][k][l+n_part]
                    
            PP[i][j][k] = max(put/np.exp(r), 0)
    
    
print 'Put option price: ', PP[0][0][0]

Put option price:  2.0162922629275823
